In [9]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_parquet("bike_data_10-19.parquet").dropna()
df1M = df.sample(1000000)
df100k = df.sample(100000)

In [32]:
tmp1 = df1M.groupby("Start_station")["ride_id"].count().reset_index()
tmp2 = df1M.groupby("End_station")["ride_id"].count().reset_index()
tmp3 = pd.merge(tmp1,tmp2,left_on="Start_station",right_on="End_station").drop(columns="End_station")
tmp3.columns = ["station","start_rides","end_rides"]
tmp3["start/end ratio"] = tmp3["start_rides"]/tmp3["end_rides"]
tmp3.sort_values("start/end ratio",ascending=True,inplace=True)
tmp3.reset_index(drop=True,inplace=True)
tmp3
px.bar(tmp3,x="station",y="start/end ratio",color="start_rides")
# most ends compared to starts: https://www.google.com/maps/place/Maple+Ave+%26+Ritchie+Ave,+Takoma+Park,+MD+20912,+USA/@38.9834543,-77.006919,167m/data=!3m1!1e3!4m5!3m4!1s0x89b7c621bbdcb88d:0x65354bd8f85c4a3b!8m2!3d38.9834543!4d-77.0065286
# most starts compared to ends: https://www.google.com/maps/place/Tenley+Cir+NW+%26+Yuma+St+NW,+Washington,+DC+20016,+USA/@38.9462929,-77.0806173,936m/data=!3m1!1e3!4m5!3m4!1s0x89b7c9c210aa4ba1:0xb82f298e96dfabe2!8m2!3d38.9462929!4d-77.0784286

In [23]:
# what to predict:
# how often is a station under peak load?
# when is a station about to be full?

,ride_id,Bike_number,Duration,Start_date,End_date,Start_station,Start_station_number,Start_capacity,End_station,End_station_number,End_capacity,start_lat,start_lon,end_lat,end_lon,Member_type
10044954,10044954,W21606,317,2015-05-30 12:09:54,2015-05-30 12:15:11,8th & F St NE,31631,23,3rd & H St NE,31616,19,38.897284,-76.994746,38.900419,-77.001951,Member
25984973,25984973,W20903,202,2019-11-25 18:22:57,2019-11-25 18:26:19,14th & R St NW,31202,19,14th & V St NW,31101,31,38.913054,-77.032010,38.917774,-77.032093,Member
730740,730740,W01089,1376,2011-06-27 17:21:30,2011-06-27 17:44:27,L'Enfant Plaza / 7th & C St SW,31218,15,Georgia Ave & Fairmont St NW,31207,15,38.886274,-77.022247,38.924911,-77.022202,Member
6714843,6714843,W20090,719,2014-05-01 09:15:31,2014-05-01 09:27:30,Lincoln Memorial,31258,25,Wilson Blvd & Ft Myer Dr,31015,19,38.888266,-77.049439,38.894605,-77.072302,Member
5432545,5432545,W20506,865,2013-09-28 11:18:52,2013-09-28 11:33:17,Adams Mill & Columbia Rd NW,31104,15,19th St & Constitution Ave NW,31235,35,38.922933,-77.042578,38.892304,-77.043597,Member
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660368,1660368,W01031,662,2012-03-01 20:03:28,2012-03-01 20:14:30,New Hampshire Ave & Ward Pl NW,31212,19,Columbia Rd & Belmont St NW,31113,19,38.905716,-77.047318,38.920680,-77.043684,Member
4045947,4045947,W00152,445,2013-04-21 17:44:33,2013-04-21 17:51:58,C & O Canal & Wisconsin Ave NW,31225,23,21st & I St NW,31205,35,38.903754,-77.062696,38.900717,-77.046450,Member
3900867,3900867,W00468,2053,2013-04-04 15:29:53,2013-04-04 16:04:07,18th St & Pennsylvania Ave NW,31242,27,Ohio Dr & West Basin Dr SW / MLK & FDR Memorials,31240,31,38.899691,-77.041538,38.884132,-77.046577,Casual
3338318,3338318,W00884,1008,2012-11-25 16:32:45,2012-11-25 16:49:33,7th & F St NW / National Portrait Gallery,31232,27,14th & D St SE,31607,19,38.897291,-77.022193,38.884068,-76.986391,Member


In [35]:
# goal: find out span of each station over time, use that as capacity
tmp1 = df[["Start_date","Start_station","End_station"]].sample(10000)
tmp1["Start_station_bikes"] = tmp1.groupby("Start_station").cumcount()
tmp1["End_station_bikes"] = (tmp1.groupby("End_station").cumcount()+1)-tmp1["Start_station_bikes"]#np.nan()
tmp1.head(50)#["End_station_bikes"] = #np.where(tmp1["Start_station"]==tmp1["End_station"], , "buy")

,Start_date,Start_station,End_station,Start_station_bikes,End_station_bikes
18541309,2017-10-10 16:57:08,15th & M St NW,Columbia Rd & Georgia Ave NW,0,1
18304134,2017-09-22 15:19:54,4th & C St SW,North Capitol St & F St NW,0,1
5252726,2013-09-09 07:22:20,12th & Irving St NE,Thomas Circle,0,1
24853016,2019-08-10 23:01:46,12th & U St NW,Thomas Circle,0,2
22986679,2019-02-11 19:11:02,Court House Metro / 15th St & N Uhle St,Barton St & 10th St N,0,1
3911445,2013-04-06 02:46:19,Massachusetts Ave & Dupont Circle NW,Jefferson Dr & 14th St SW,0,1
9611245,2015-04-22 12:08:19,Glebe Rd & 11th St N,Langston Blvd & N Monroe St,0,1
9572230,2015-04-18 17:51:31,34th & Water St NW,17th & K St NW,0,1
1924604,2012-04-18 07:29:28,14th & Harvard St NW,14th & Rhode Island Ave NW,0,1
16588871,2017-05-10 13:52:45,34th & Water St NW,Lincoln Memorial,1,0
